<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jayalamar_book_chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

tomatoes = load_dataset('rotten_tomatoes')
train_data, test_data = tomatoes['train'],tomatoes['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

# Supervised Classification

**Hugging Face Trainer**

In [6]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
  "Tokenize the input data"
  return tokenizer(examples['text'],truncation = True)

tokenized_train = train_data.map(preprocess_function,batched = True)
tokenized_test = test_data.map(preprocess_function, batched = True)



Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00


In [11]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  logits,labels = eval_pred
  predictions = np.argmax(logits,axis = -1)
  load_f1 = evaluate.load('f1')
  f1 = load_f1.compute(predictions = predictions, references = labels)['f1']
  return {'f1':f1}


In [16]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = 'model',
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    num_train_epochs = 1,
    weight_decay = 0.01,
    save_strategy = 'epoch',
    report_to = "none"
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics

)

<ipython-input-16-853703f8506a>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=267, training_loss=0.40972143166074143, metrics={'train_runtime': 4509.4309, 'train_samples_per_second': 1.892, 'train_steps_per_second': 0.059, 'total_flos': 246561781088880.0, 'train_loss': 0.40972143166074143, 'epoch': 1.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.3604084849357605,
 'eval_f1': 0.8515037593984962,
 'eval_runtime': 215.0432,
 'eval_samples_per_second': 4.957,
 'eval_steps_per_second': 0.158,
 'epoch': 1.0}